In [91]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import random

In [92]:
!pip install bs4

In [93]:
"""Extracts HTML from a webpage""" 
def scrape_page(page_url):
    answer = requests.get(page_url)
    soup = BeautifulSoup(answer.content, features='html.parser')
    return soup

In [94]:
"""
Return soup object from url_obj 
where url_obj is:
{page_num <int> : href <str>}
e.g {1: "www.airbnb.com/1/"}

"""
def extract_listing(url_obj):
    page_url = ""
    pag_key = ""
    for key in url_obj:
        page_url = url_obj[key]
        page_key = key
    """Extracts listings from an Airbnb search page"""
    page_soup = scrape_page(page_url)
    formatted_date =  datetime.fromtimestamp(datetime.now().timestamp())
    print("Extracted Data: {}--{}\t".format(page_url, formatted_date))
    return page_soup

In [95]:
soup = extract_listing({1: "https://www.airbnb.com/s/New-York/homes?place_id=ChIJOwg_06VPwokRYv534QaPC8g&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=november&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_dates%5B%5D=june&flexible_trip_dates%5B%5D=july&flexible_trip_dates%5B%5D=august&flexible_trip_dates%5B%5D=september&flexible_trip_dates%5B%5D=october&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=flexible_dates&adults=0&children=0&infants=0&pets=0&search_type=AUTOSUGGEST"})

Extracted Data: https://www.airbnb.com/s/New-York/homes?place_id=ChIJOwg_06VPwokRYv534QaPC8g&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=november&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_dates%5B%5D=june&flexible_trip_dates%5B%5D=july&flexible_trip_dates%5B%5D=august&flexible_trip_dates%5B%5D=september&flexible_trip_dates%5B%5D=october&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=flexible_dates&adults=0&children=0&infants=0&pets=0&search_type=AUTOSUGGEST--2022-11-29 03:12:51.950230	


In [96]:
"""
soup: page soup object
className: attribute id that has anchor tag
count: page number count we are currently extracting
"""
def extract_anchors(soup, className, count=0):       
    processed_classes = process_anchor(className, soup.find_all("a", {"class": className}), count) 
    print("Done extracting anchor {}".format(count))
    return processed_classes

def process_anchor(className, anchors, count=0):
    overall_data = {}
    for anchor in anchors:
        text, data = get_anchor_data(anchor) # Get anchor page number<str> and href value<str>
        if text == None and data == None:
            continue
        if count > 0 and text == str(count):
            overall_data[text] = data
        if count == 0:
            overall_data[text] = data
    print("OVERALL_DATA", overall_data)
    return overall_data

def get_anchor_data(anchor):
    anchor_text = anchor.get_text()
    if len(anchor_text) != 0 and anchor_text != "15":
        text = anchor.get_text()
        data = anchor.get("href")
        return text, data
    return None, None

In [97]:
class_name = "_833p2h"
anchor_data = extract_anchors(soup, class_name)

OVERALL_DATA {'2': '/s/New-York/homes?place_id=ChIJOwg_06VPwokRYv534QaPC8g&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=august&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=july&flexible_trip_dates%5B%5D=june&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=may&flexible_trip_dates%5B%5D=november&flexible_trip_dates%5B%5D=october&flexible_trip_dates%5B%5D=september&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=flexible_dates&search_type=AUTOSUGGEST&tab_id=home_tab&query=New%20York&price_filter_input_type=0&price_filter_num_nights=2&federated_search_session_id=9daf9f2e-0a97-471c-9dd1-0b91559e4e44&pagination_search=true&cursor=eyJzZWN0aW9uX29mZnNldCI6MiwiaXRlbXNfb2Zmc2V0IjoyMCwidmVyc2lvbiI6MX0%3D', '3': '/s/New-York/homes?place_id=ChIJOwg_06VPwokRYv534QaPC8g&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%

In [98]:
"""
Construct list of url objects containing url_obj
{page_num<int>: href<str>}

"""
def get_urls(anchor_data, urls):
    base_url = "https://www.airbnb.com"
    for key, value in anchor_data.items():
        url_obj = {}
        url_obj[key] = str(base_url + anchor_data[key]) 
        urls.append(url_obj)
    return urls

In [99]:
urls_list = [{1: "https://www.airbnb.com/s/New-York/homes?place_id=ChIJOwg_06VPwokRYv534QaPC8g&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=november&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_dates%5B%5D=june&flexible_trip_dates%5B%5D=july&flexible_trip_dates%5B%5D=august&flexible_trip_dates%5B%5D=september&flexible_trip_dates%5B%5D=october&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=flexible_dates&adults=0&children=0&infants=0&pets=0&search_type=AUTOSUGGEST"}] 
urls = get_urls(anchor_data, urls_list)
urls

[{1: 'https://www.airbnb.com/s/New-York/homes?place_id=ChIJOwg_06VPwokRYv534QaPC8g&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=november&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_dates%5B%5D=june&flexible_trip_dates%5B%5D=july&flexible_trip_dates%5B%5D=august&flexible_trip_dates%5B%5D=september&flexible_trip_dates%5B%5D=october&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=flexible_dates&adults=0&children=0&infants=0&pets=0&search_type=AUTOSUGGEST'},
 {'2': 'https://www.airbnb.com/s/New-York/homes?place_id=ChIJOwg_06VPwokRYv534QaPC8g&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=august&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=july&flexible_trip_dates%5B%5D=june&fl

In [100]:
# Custom function to get url pages 5-14
def get_remaining_urls(urls):
    count = 5
    class_name=  "_833p2h"
    url_data = [urls[-1]]
    while count <= 15:
        count_soup = extract_listing(url_data[-1])
        count_anchor = extract_anchors(count_soup, class_name, count)
        url_data = get_urls(count_anchor, url_data)
        count+=1
    return url_data

In [101]:
remaining_urls = get_remaining_urls(urls)

Extracted Data: https://www.airbnb.com/s/New-York/homes?place_id=ChIJOwg_06VPwokRYv534QaPC8g&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=august&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=july&flexible_trip_dates%5B%5D=june&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=may&flexible_trip_dates%5B%5D=november&flexible_trip_dates%5B%5D=october&flexible_trip_dates%5B%5D=september&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=flexible_dates&search_type=AUTOSUGGEST&tab_id=home_tab&query=New%20York&price_filter_input_type=0&price_filter_num_nights=2&federated_search_session_id=9daf9f2e-0a97-471c-9dd1-0b91559e4e44&pagination_search=true&cursor=eyJzZWN0aW9uX29mZnNldCI6MiwiaXRlbXNfb2Zmc2V0Ijo2MCwidmVyc2lvbiI6MX0%3D--2022-11-29 03:12:53.852610	
OVERALL_DATA {'5': '/s/New-York/homes?place_id=ChIJOwg_06VPwokRYv534QaPC8g&refinement_paths%5B%5D=%2

In [102]:
all_urls = urls + remaining_urls[1:]
all_urls

[{1: 'https://www.airbnb.com/s/New-York/homes?place_id=ChIJOwg_06VPwokRYv534QaPC8g&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=november&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_dates%5B%5D=june&flexible_trip_dates%5B%5D=july&flexible_trip_dates%5B%5D=august&flexible_trip_dates%5B%5D=september&flexible_trip_dates%5B%5D=october&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=flexible_dates&adults=0&children=0&infants=0&pets=0&search_type=AUTOSUGGEST'},
 {'2': 'https://www.airbnb.com/s/New-York/homes?place_id=ChIJOwg_06VPwokRYv534QaPC8g&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=august&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=july&flexible_trip_dates%5B%5D=june&fl

In [103]:
# Extract soup objects and query for div that has listing data
listing_pages = [extract_listing(urlObj).find_all("div", {"class": "g1qv1ctd cb4nyux dir dir-ltr"}) for urlObj in all_urls]

Extracted Data: https://www.airbnb.com/s/New-York/homes?place_id=ChIJOwg_06VPwokRYv534QaPC8g&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=november&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_dates%5B%5D=june&flexible_trip_dates%5B%5D=july&flexible_trip_dates%5B%5D=august&flexible_trip_dates%5B%5D=september&flexible_trip_dates%5B%5D=october&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=flexible_dates&adults=0&children=0&infants=0&pets=0&search_type=AUTOSUGGEST--2022-11-29 03:13:13.144486	
Extracted Data: https://www.airbnb.com/s/New-York/homes?place_id=ChIJOwg_06VPwokRYv534QaPC8g&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=august&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=january&flexible_trip_dates%

In [104]:
def get_df_data(listing_pages):
    df_data = list()
    for listings in listing_pages:
    # Containers used to store div data, clear for each listing
        a = []
        b = []
        c = []
        d = []
        e = []
        f = []
        for listing in listings:
            listing_data = extract_listing_data(listing) # Use helper func to map soup -> obj
            a.append(listing_data["title"])
            b.append(listing_data["description"])
            c.append(listing_data["num_bed"])
            d.append(listing_data["price_val"])
            e.append(listing_data["price_per_night"])
            f.append(listing_data["average_rating_num_rating"])
        df_data.append({
            "title": a,
            "description": b,
            "num_beds": c ,
            "price_val": d ,
            "price_per_night": e,
            "average_rating_num_rating": f
        })
        print("Processed soup: {}".format(len(df_data)))
    return df_data

# Helper function for mapping soup data to obj for row data
def extract_listing_data(listing):
    listing_data = {}
    listing_data["title"] = listing.find("div", {"class": "t1jojoys dir dir-ltr"}).contents[0] # title
    listing_data["description"] = listing.find("span", {"class": "t6mzqp7 dir dir-ltr"}).contents[0] # Span description
    listing_data["num_bed"] = listing.find("span", {"class": "dir dir-ltr"}).contents[0] # div num beds
    listing_data["price_val"] = listing.find("span", {"class": "_tyxjp1"}).contents[0] # price_val
    price_per_night_data = listing.find("span", {"class": "a8jt5op dir dir-ltr"}).contents[0] # price per night
    listing_data["price_per_night"] = " ".join(str(s) for s in price_per_night_data)
    listing_data["average_rating_num_rating"] = listing.find("span", {"class": "r1dxllyb dir dir-ltr"}).contents[0] if listing.find("span", {"class": "r1dxllyb dir dir-ltr"}) else "NONE"  # Average rating and num of ratings
    return listing_data

In [105]:
df_data = get_df_data(listing_pages) 

Processed soup: 1
Processed soup: 2
Processed soup: 3
Processed soup: 4
Processed soup: 5
Processed soup: 6
Processed soup: 7
Processed soup: 8
Processed soup: 9
Processed soup: 10
Processed soup: 11
Processed soup: 12
Processed soup: 13
Processed soup: 14


In [106]:
count=1
for data in df_data:
    file_name = "data__{}__{}.csv".format(count, random.getrandbits(128))
    pd.DataFrame(data).to_csv(file_name)
    print("Generated_file: {}".format(file_name))
    count+=1

Generated_file: data__1__270860741414586709426283340062031621717.csv
Generated_file: data__2__186999598391259057149330180195200427983.csv
Generated_file: data__3__120983156241965321590163828777464058554.csv
Generated_file: data__4__228163983433688185952852931588281879306.csv
Generated_file: data__5__326214118082054448453583362595960104745.csv
Generated_file: data__6__187986097088981747224690803069661565001.csv
Generated_file: data__7__70338870891707727269483338425088057373.csv
Generated_file: data__8__133160643279053423527211333975808419055.csv
Generated_file: data__9__227529891547171791158852419794901353161.csv
Generated_file: data__10__296891250446986191704287717795135204444.csv
Generated_file: data__11__233029790877004480662260083310259350150.csv
Generated_file: data__12__191398689958276005860126157701578746695.csv
Generated_file: data__13__158754169656107556163436600876435251823.csv
Generated_file: data__14__279676489565611684542128369483964027956.csv
